In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from sklearn.datasets import load_digits
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import numpy as np
import random

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# データをロードします
digits = load_digits()
data = digits.data
targets = digits.target

# データを正規化します
data = data / 16.0  # 0-16までの整数値なので16で割って正規化

# NumPy配列をPyTorchのテンソルに変換します
data = torch.tensor(data, dtype=torch.float32)
targets = torch.tensor(targets, dtype=torch.int64)

# TensorDatasetとDataLoaderを構築
dataset = TensorDataset(data, targets)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, latent_dim):
        super(VAE, self).__init__()
        # エンコーダー: 入力層 -> 隠れ層1 -> 隠れ層2 -> 平均と標準偏差の線形層
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim1),
            nn.ReLU(),
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.ReLU()
        )
        
        self.fc_mu = nn.Linear(hidden_dim2, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim2, latent_dim)
        
        # デコーダー: 潜在空間 -> 隠れ層1 -> 隠れ層2 -> 出力層
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim2),
            nn.ReLU(),
            nn.Linear(hidden_dim2, hidden_dim1),
            nn.ReLU(),
            nn.Linear(hidden_dim1, input_dim),
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

def loss_function(recon_x, x, mu, logvar):
    # バイナリ クロスエントロピ 損失と KL ダイバージェンス
    BCE = nn.functional.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD
    
# モデルの設定
input_dim = data.shape[1]  # 64 (8x8画像)
hidden_dim1 = 128
hidden_dim2 = 64
latent_dim = 2

# モデル、損失関数、最適化手法の設定
model = VAE(input_dim, hidden_dim1, hidden_dim2, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 学習を行います
num_epochs = 100
losses = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_data, _ in dataloader:
        optimizer.zero_grad()
        reconstructed, mu, logvar = model(batch_data)
        loss = loss_function(reconstructed, batch_data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

    average_loss = train_loss / len(dataloader.dataset)
    
    losses.append(average_loss)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}')

# 損失のプロット
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss per Epoch')
plt.legend()
plt.grid(True)
plt.show()

# 全データを用いて潜在変数を取得してプロット
model.eval()
with torch.no_grad():
    mu, logvar = model.encode(data)
    latent_space = model.reparameterize(mu, logvar).numpy()

plt.figure(figsize=(8, 6))
plt.scatter(latent_space[:, 0], latent_space[:, 1], alpha=0.5, c=targets, cmap='tab10')
plt.colorbar()
plt.title('Latent Space')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.show()

In [ ]:
# モデルを評価モードに変更
model.eval()
with torch.no_grad():
    # 先頭10件の訓練データを選択（X_trainはすでにTensorとして定義済み）
    sample_data = data[:10]
    # エンコーダを通して mu と logvar を取得
    mu, logvar = model.encode(sample_data)
    # sigma^2 を計算（σ^2 = exp(log σ^2)）
    sigma_squared = torch.exp(logvar)
    
    print("Selected training samples sigma^2:")
    print(sigma_squared)


In [ ]:
import matplotlib.pyplot as plt

# 訓練済みモデルを評価モードに切り替え
model.eval()

# 入力データをモデルに通して再構成を取得
with torch.no_grad():
    reconstructed_data, _ , _ = model(data)

# Tensor を numpy array に変換
input_data_np = data.numpy()
reconstructed_data_np = reconstructed_data.numpy()

# 比較するための画像数を設定
num_images_to_show = 10  # 例えば最初の10個を表示

# 図示
fig, axes = plt.subplots(nrows=2, ncols=num_images_to_show, figsize=(15, 4))
for i in range(num_images_to_show):
    # 入力画像
    axes[0, i].imshow(input_data_np[i].reshape(8, 8), cmap='gray')
    axes[0, i].axis('off')
    
    # 再構成画像
    axes[1, i].imshow(reconstructed_data_np[i].reshape(8, 8), cmap='gray')
    axes[1, i].axis('off')

axes[0, 0].set_title("Original", fontsize=12)
axes[1, 0].set_title("Reconstructed", fontsize=12)

plt.show()

In [ ]:
# ユーザー追加

factor = 0.5
(x1, y1), (x2, y2) = (latent_space.min(axis=0) - latent_space.ptp(axis=0)*factor,
                      latent_space.max(axis=0) + latent_space.ptp(axis=0)*factor)

print("xrange", x1,x2)
print("yrange", y1,y2)

In [ ]:
import numpy as np

# 潜在空間を10x10のグリッドに分割
x_range = (x1, x2)
y_range = (y2, y1)

# 分割数
n_bins = 10

# 潜在空間を10x10のグリッドに分割
fig, axarr = plt.subplots(n_bins, n_bins, figsize=(15, 15))

x_bins = np.linspace(x_range[0], x_range[1], n_bins)
y_bins = np.linspace(y_range[0], y_range[1], n_bins)

# 各グリッドに対応する潜在コードからデコードされた各画像を表示
for i, x_val in enumerate(x_bins):
    for j, y_val in enumerate(y_bins):
        # 選ばれた潜在変数のグリッドでの座標
        z = torch.tensor([[x_val, y_val]], dtype=torch.float32)

        # デコードして画像を生成
        with torch.no_grad():
            generated = model.decoder(z).numpy().reshape(8, 8)

        # 画像をプロット
        axarr[j, i].imshow(generated, cmap='gray')
        axarr[j, i].axis('off')

plt.suptitle('Decoding the Latent Space Grid', fontsize=16)
plt.subplots_adjust(wspace=0.1, hspace=0.1)
plt.show()